# 1. Importação das bilbiotecas necessárias


In [2]:
# Importando o pandas
import pandas as pd

# Importando biblioteca para trabalhar com o MongoDB
import pymongo as pym

# Importando o numpy para trabalhar com cálculos matemáticos
import numpy as np

#Importando o seaborn e o matplotlib para análises gráficas
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Dados dos Alunos

In [3]:
# Como o arquivo dos alunos é maior, 
#a seleção das colunas utilizadas será feita  antes da importação do csv
colunas = ['CO_IES','CO_CURSO','ID_ALUNO','CO_ALUNO_CURSO',
          'TP_COR_RACA','TP_SEXO','NU_IDADE','TP_NACIONALIDADE','IN_DEFICIENCIA',
          'TP_SITUACAO','IN_INGRESSO_ENEM','IN_FINANCIAMENTO_ESTUDANTIL',
           'IN_FIN_NAOREEMB_PROUNI_PARCIAL','IN_FIN_NAOREEMB_PROUNI_INTEGR',
          'TP_ESCOLA_CONCLUSAO_ENS_MEDIO','IN_MOBILIDADE_ACADEMICA']

## 2.1 Pegando os alunos por sexo

In [4]:
array_df = []
for chunk in pd.read_csv('../data/SUP_ALUNO_2019.CSV', sep="|", encoding="latin-1", usecols=colunas, chunksize = 10000):
    temp_df = chunk.loc[chunk['TP_SITUACAO'] == 3]
    array_df.append(temp_df)

alunos = pd.concat(array_df, ignore_index=True)


## alunos = pd.read_csv('../data/SUP_ALUNO_2019.CSV', sep="|", encoding="latin-1", usecols=colunas)

In [ ]:
# Verificando a quantidade de linhas e colunas do arquivo 
alunos.shape

In [ ]:
# Verificando as 5 primeiras linhas do arquivo
alunos.head()

In [ ]:
# Verificando o tipo de cada coluna
alunos.dtypes

## 2.2  Tratando os dados do dataset

In [ ]:
## Verificando a quantidade de valores nulos por coluna
alunos.isnull().sum()

In [ ]:
# De financiamento para 2, para depois alterar o 2 para "Não informado"
alunos['IN_FINANCIAMENTO_ESTUDANTIL'].fillna(value='2', inplace=True)
alunos['IN_FIN_NAOREEMB_PROUNI_INTEGR'].fillna(value='2', inplace=True)
alunos['IN_FIN_NAOREEMB_PROUNI_PARCIAL'].fillna(value='2', inplace=True)

# Alterando para 0 os mobilidade acadêmica nulos
alunos['IN_MOBILIDADE_ACADEMICA'].fillna(value='0', inplace=True)

# 3 Dados das IES

In [ ]:
# Como cada linha do arquivo de alunos é vinculado a uma IES via código da IES, é necessário
# realizar a carga dos dados das IES e inserir em cada aluno 

# 3.1 Carga do arquivo e dados inicial

In [ ]:
# Lendo o arquivo CSV disponível nos microdados do Censo da Educação Superior e gravando
# ele em um DataFrame
ies = pd.read_csv('../data/SUP_IES_2019.CSV', sep="|", encoding="latin-1")

In [ ]:
# Mostrando a quantidade de linhas e colunas do DataFrame 
print("Quantidade de linhas:",ies.shape[0])
print("Quantidade de colunas:",ies.shape[1])

In [ ]:
# Importando arquivo de cidades do IBGE que servirá para popular o endereço das IES
colunas = ['codigo_ibge','NO_MUNICIPIO','LAT_MUNICIPIO','LON_MUNICIPIO','CO_UF','SG_UF','NO_UF']
municipios = pd.read_csv('../data/municipios.csv', sep=",", header=0, names=colunas)

In [ ]:
# Removendo colunas desnecessárias
municipios.drop(['CO_UF'], axis=1, inplace = True)

## 3.2 Tratando os dados do dataset

In [ ]:
# Criando um DataFrame somente com as colunas de endereço das IES 
# E outro com as colunas consideradas na estrutura do trabalho
endereco_ies = ies[{'CO_REGIAO','CO_UF','CO_MUNICIPIO','IN_CAPITAL'}]

ies = ies[{'CO_IES','NO_IES','SG_IES',
           'TP_CATEGORIA_ADMINISTRATIVA',
           'TP_ORGANIZACAO_ACADEMICA','QT_TEC_TOTAL'}]

In [ ]:
# Adicionando uma coluna Endereco no DataFrame das IES, pois por padrão
# essa coluna não existe
ies['ENDERECO'] = ""

In [ ]:
## Categorizando o IN_CAPITAL do endereco 
# 1) Alterando o tipo dos dados int, para object (string)
endereco_ies['IN_CAPITAL'] = endereco_ies['IN_CAPITAL'].apply(str)

In [ ]:
# Alterando os valores por suas respectivas categorias  
# 2) IN CAPITAL 0: Não; 1: Sim. 
endereco_ies.loc[endereco_ies['IN_CAPITAL'] == '0', 'IN_CAPITAL'] = 'Não'
endereco_ies.loc[endereco_ies['IN_CAPITAL'] == '1', 'IN_CAPITAL'] = 'Sim'

In [ ]:
# fazendo um merge de endereço com municipios
endereco_ies = endereco_ies.merge(municipios, how='left', left_on='CO_MUNICIPIO', right_on='codigo_ibge')

In [ ]:
# Removendo colunas desnecessárias
endereco_ies.drop(['codigo_ibge'], axis=1, inplace = True)

In [ ]:
# Resultado da junção de endereço e municipios
endereco_ies.head(3)

In [ ]:
endereco_ies.isnull().sum()

In [ ]:
endereco_ies.shape

In [ ]:
# Criando um dicionário do Python a partir do DataFrame de enderecos
endereco_ies_dict = endereco_ies.to_dict('records')

In [ ]:
# Adicionando um endereco (através do dicionário criado no passo anterior)
# e inserindo na coluna "ENDERECO" de cada IES
for i in ies.index:
    ies.at[i,'ENDERECO'] = endereco_ies_dict[i]  

In [ ]:
ies.isnull().sum()

In [ ]:
# Alterando os valores nulos da sigla da IES para 'Sem sigla'.
ies['SG_IES'].fillna('Sem SIGLA', inplace = True)

In [ ]:
# 1) Alterando o tipo dos dados int, para object (string)
ies['TP_CATEGORIA_ADMINISTRATIVA'] = ies['TP_CATEGORIA_ADMINISTRATIVA'].apply(str)
ies['TP_ORGANIZACAO_ACADEMICA'] = ies['TP_ORGANIZACAO_ACADEMICA'].apply(str)


In [ ]:
# Alterando os valores por suas respectivas categorias  
# 2) TP_CATEGORIA_ADMINISTRATIVA: 1: Pública Federal; 2: Pública Federal; 
# 3: Pùblica Municipal; 4: Privada com fins lucrativos; 5: Privada sem fins lucrativos;
# 6: Privada - Particular em sentido estrito; 7: Especial
# 8: Privada comunitária; 9: Privada confessional.  
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '1', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Pública Federal'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '2', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Pública Federal'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '3', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Pública Municipal'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '4', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Privada com fins lucrativo'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '5', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Privada sem fins lucrativos'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '6', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Privada - Particular em sentido estrito'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '7', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Especial'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '8', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Privada comunitária'
ies.loc[ies['TP_CATEGORIA_ADMINISTRATIVA'] == '9', 'TP_CATEGORIA_ADMINISTRATIVA'] = 'Privada confessional'

# 2) TP_ORGANIZACAO_ACADEMICA: 1: Universidade; 2: Centro Universitário; 
# 3: Faculdade; 4: Instituto Federal de Educação Ciência e Tecnologia; 
# 5: Centro Federal de Educação Tecnológica
  
ies.loc[ies['TP_ORGANIZACAO_ACADEMICA'] == '1', 'TP_ORGANIZACAO_ACADEMICA'] = 'Universidade'
ies.loc[ies['TP_ORGANIZACAO_ACADEMICA'] == '2', 'TP_ORGANIZACAO_ACADEMICA'] = 'Centro Universitário'
ies.loc[ies['TP_ORGANIZACAO_ACADEMICA'] == '3', 'TP_ORGANIZACAO_ACADEMICA'] = 'Faculdade'
ies.loc[ies['TP_ORGANIZACAO_ACADEMICA'] == '4', 'TP_ORGANIZACAO_ACADEMICA'] = 'Instituto Federal de Educação Ciência e Tecnologia'
ies.loc[ies['TP_ORGANIZACAO_ACADEMICA'] == '5', 'TP_ORGANIZACAO_ACADEMICA'] = 'Centro Federal de Educação Tecnológica'


# 4 Juntando o DataSet de Alunos com o de IES

In [ ]:
alunos = alunos.merge(ies, how='left', left_on='CO_IES', right_on='CO_IES')

In [ ]:
alunos.head()

In [ ]:
alunos.isnull().sum()

In [ ]:
alunos_ies = alunos[{'CO_IES','TP_CATEGORIA_ADMINISTRATIVA','QT_TEC_TOTAL','NO_IES','SG_IES','TP_ORGANIZACAO_ACADEMICA','ENDERECO'}]

In [ ]:
alunos.drop({'CO_IES','TP_CATEGORIA_ADMINISTRATIVA','QT_TEC_TOTAL','NO_IES','SG_IES','TP_ORGANIZACAO_ACADEMICA','ENDERECO'}, axis=1, inplace = True)

In [ ]:
alunos['IES'] = ""

In [ ]:
# Criando um dicionário do Python a partir do DataSet de docentes_ies
alunos_ies_dict = alunos_ies.to_dict('records')

In [ ]:
# Adicionando uma ies (através do dicionário criado no passo anterior)
# e inserindo na coluna "ENDERECO" de cada aluno
for x in alunos.index:
    alunos.at[x,'IES'] = alunos_ies_dict[x]  

In [ ]:
alunos.shape

In [ ]:
alunos.head(2)

# 5 Dados de Cursos

#  5.1 Carga do arquivo e dados iniciais 

In [1]:
# Lendo o arquivo CSV disponível nos microdados do Censo da Educação Superior e gravando
# ele em um DataFrame
cursos = pd.read_csv('../data/SUP_CURSO_2019.CSV', sep="|", encoding="latin-1")

NameError: name 'pd' is not defined

# 6. Envio dos dados para o mongoDB

In [ ]:
# Criando a conexão com o mongoDB
client = pym.MongoClient('mongodb://localhost:27017/')

# Acessando a base de dados criada para o teste
db = client['censo']

# Acessando a coleção curso dentro da base de dados censo  
collection_cursos=  db['alunos']

In [ ]:
# Transformando o DataSet dos cursos em um dicionario com o objetivo
# de salvar ele no MongoDB, pois o banco não aceita estruturas do tipo DataFrame
alunos_dict = alunos.to_dict('records')

In [ ]:
# Enviando os dados para o MongoDB
collection_cursos.insert_many(alunos_dict)